In [1]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
# Path to chrome driver
ChromeDriver = "/Users/huanglinchun/Desktop/VS Code Files/chromedriver_mac64/chromedriver.exe"

# Call the webdriver
driver = webdriver.Chrome(ChromeDriver)

# Source: Alphaspread.com
url = 'https://www.alphaspread.com/security/nasdaq/aapl/financials/earnings-call-transcripts'
driver.get(url)
sleep(2)

######################################################################################################
#  Scrape AAPL earnings call transcripts from 2013Q1 to 2022Q4
# (2023Q1, 2023Q2 are downloaded seperatedly from : https://capedge.com/transcript/320193/2023Q2/AAPL)

# TODO : get all dates of the earnings call
soup = BeautifulSoup(driver.page_source, 'html.parser')
dates = []
date = soup.find_all('div', class_='detail no-weight')
DateRegex = re.compile(r'\d\d\d\d-\d\d-\d\d')
for _ in date:
    to_append = DateRegex.findall(_.text)
    dates.append(to_append[0])

# TODO : web scraping loop
index = 0
for times in range(40):
    
    # Get the earnings call title
    title = driver.find_element(By.XPATH, '//*[@id="main"]/div[2]/div[2]/div/div/div/div[2]/div/div[1]').text

    # Get web parser
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Get the content of transcript
    transcript = soup.find('div', class_='earnings-call-transcript')
    paragraphs = transcript.find_all('div', class_='comment')

    # Save
    with open(f'transcript_{title}_{dates[index]}.txt', 'w') as file:
        for _ in paragraphs:
            tex = _.find('div', class_='content').text
            file.write(tex)
    file.close()
    
    # click the "next" buttom 
    next_buttom = driver.find_element(By.XPATH, '//*[@id="main"]/div[2]/div[2]/div/div/div/div[2]/div/div[1]/div[2]')
    next_buttom.click()

    # Index + 1 (To update the date of transcript)
    index += 1
    sleep(1)

/var/folders/gl/stf2yn_54sdd_k_jlfmc4nvr0000gn/T/ipykernel_42413/3198519056.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriver)


In [13]:
import os
directory = '/Users/huanglinchun/Desktop/VS Code Files/AAPL_Earnings_Call_Text_Analysis/transcripts'
pattern = r', '
replacement = '_'
for filename in os.listdir(directory):
    # Rename files here
    new_name = re.sub(pattern, replacement, filename)  # Replace with the desired new file name
    
    os.rename(os.path.join(directory, filename), os.path.join(directory, new_name))

